In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
import requests

drive.mount('/content/gdrive')

%cd "/content/gdrive/My Drive/CSE551 Group Projects/Final Project Presentation"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/.shortcut-targets-by-id/1zLdF-ovpoDMI1rA3kRe4yk1TTgZbQWYk/CSE551 Group Projects/Final Project Presentation


In [ ]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.2 MB/s eta 0:00:00


In [ ]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 40.9256,
	"longitude": -73.1409,
	"start_date": "2024-02-01",
	"end_date": "2024-03-31",
	"hourly": "temperature_2m",
	"temperature_unit": "fahrenheit",
	"timezone": "America/New_York"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 40.949031829833984°N -73.076904296875°E
Elevation 28.0 m asl
Timezone b'America/New_York' b'EDT'
Timezone difference to GMT+0 -14400 s
                          date  temperature_2m
0    2024-02-01 04:00:00+00:00       31.053200
1    2024-02-01 05:00:00+00:00       30.783199
2    2024-02-01 06:00:00+00:00       30.693199
3    2024-02-01 07:00:00+00:00       30.603199
4    2024-02-01 08:00:00+00:00       30.693199
...                        ...             ...
1435 2024-03-31 23:00:00+00:00       53.643200
1436 2024-04-01 00:00:00+00:00       51.213200
1437 2024-04-01 01:00:00+00:00       49.323200
1438 2024-04-01 02:00:00+00:00       49.053200
1439 2024-04-01 03:00:00+00:00       47.973198

[1440 rows x 2 columns]


In [ ]:
data = pd.read_csv('TwoMonths.csv')

print(data)
hourly_consumption = 0

new_data = {
    'Start': [],
    'kWh': [],
    'temperature': []
}
counter = 0

for index, row in data.iterrows():
  hourly_consumption += row["kWh"]
  if(index % 2 == 1):
    new_data['Start'].append(row["Start"][:row["Start"].index(":", 13)] + row["Start"][row["Start"].index(":", 13)+3:])
    new_data['kWh'].append(hourly_consumption)
    new_data['temperature'].append(str(hourly_data['temperature_2m'][counter])[:5])
    counter += 1
    hourly_consumption = 0

filtered_data = pd.DataFrame(data = new_data)
print(filtered_data)

                      Start             Meter        Location   kWh
0       2/1/2024 4:00:00 AM   Meter #80354166  85230256077175  1.70
1       2/1/2024 4:00:00 AM  Meter #80354166g  85230256077175  0.00
2       2/1/2024 5:00:00 AM   Meter #80354166  85230256077175  0.87
3       2/1/2024 5:00:00 AM  Meter #80354166g  85230256077175  0.00
4       2/1/2024 6:00:00 AM   Meter #80354166  85230256077175  0.59
...                     ...               ...             ...   ...
2867   3/31/2024 9:00:00 PM  Meter #80354166g  85230256077175  0.00
2868  3/31/2024 10:00:00 PM   Meter #80354166  85230256077175  1.10
2869  3/31/2024 10:00:00 PM  Meter #80354166g  85230256077175  0.00
2870  3/31/2024 11:00:00 PM   Meter #80354166  85230256077175  0.77
2871  3/31/2024 11:00:00 PM  Meter #80354166g  85230256077175  0.00

[2872 rows x 4 columns]
                   Start   kWh temperature
0       2/1/2024 4:00 AM  1.70       31.05
1       2/1/2024 5:00 AM  0.87       30.78
2       2/1/2024 6:00 AM  0.59

In [ ]:
filtered_data.to_csv('filtered_data.csv', index=False)